<a href="https://colab.research.google.com/github/carsofferrei/04_data_processing/blob/main/spark/examples/02-dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark/examples/02-dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataframe
-

# Setting up PySpark

In [1]:
%pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').config('spark.ui.port', '4050').getOrCreate()
sc = spark.sparkContext

# DataFrames
- Untyped Datasets
- Similar to tables in relational databases
- DataFrames are just Dataset of Rows in Scala and Java API. These operations are also referred as “untyped transformations” in contrast to “typed transformations” come with strongly typed Scala/Java Datasets.

In [8]:
# Creating from RDDs
# .toDF()

lst = [("c1", "v1"), ("c2", "v2"), ("c3", "v3")]

# transformations
rdd = sc.parallelize(lst)
df = rdd.toDF(["col", "value"]) #tem que ser uma lista para definir as colunas

# action
df.show()
df.take(1)
#df.head(1)



[Row(col='c1', value='v1')]

In [9]:
# Using "createDataFrame" + StructTypes

from pyspark.sql.types import *
# fazendo * estamos a importar tudo o que esta na classe, mas a boa prática é garantir que só importamos o que é necessário

data = [("c1", "v1"), ("c2", "v2"), ("c3", "v3")]

# definição do schema
schema = StructType([
    StructField("col", StringType(), True),
    StructField("value", StringType(), True)
])

# temos sempre que passar a coleção de dados e o schema
df = spark.createDataFrame(data, schema=schema)
df.show()


+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [10]:
# Read data from data sources
https://spark.apache.org/docs/3.5.2/sql-data-sources.html


SyntaxError: invalid syntax (<ipython-input-10-a3e1ec46bf46>, line 2)

In [11]:
!mkdir -p /content/files/samples/

In [12]:
# from csv
import csv

# creating csv file
with open('/content/files/samples/file.csv', 'w', newline='') as csvfile:
    fieldnames = ['col', 'value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=";")
    writer.writeheader()
    writer.writerow({'col': 'c1', 'value': 'v1'})
    writer.writerow({'col': 'c2', 'value': 'v2'})
    writer.writerow({'col': 'c3', 'value': 'v3'})

# read csv file
df = spark.read.format("csv").load("/content/files/samples/file.csv", sep=";", header=True)
df.show()

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [13]:

# from json
json = """[{"col": "c1", "value": "v1"}, {"col": "c2", "value": "v2"}, {"col": "c3", "value": "v3"}]"""

text_file = open("/content/files/samples/file.json", "w")
text_file.write(json)
text_file.close()

# read from json (dá para fazer em todos os formatos de 2 formas)
df = spark.read.format("json").load("/content/files/samples/file.json")
df = spark.read.json("/content/files/samples/file.json")
df.show()

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [14]:
# from parquet

# Generating parquet
lst = [("c1", "v1"), ("c2", "v2"), ("c3", "v3")]
rdd = sc.parallelize(lst)
df = rdd.toDF(["col", "value"])

# Dá para fazer "overwrite" ou "append". O overwrite elimina completamente o que existia nessa pasta e escreve de novo.
# O append incrementa.
df.write.format("parquet").mode("overwrite").save("/content/files/samples/parquet")

# Origina serem guardados 2 ficheiros
# df = df.repartition(2)

# read from parquet
df2 = spark.read.format("parquet").load("/content/files/samples/parquet")
df2.show()

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [15]:
# Check schema
df.printSchema()
print(df.schema)
print(df.columns)

root
 |-- col: string (nullable = true)
 |-- value: string (nullable = true)

StructType([StructField('col', StringType(), True), StructField('value', StringType(), True)])
['col', 'value']


In [16]:
# counting items in the dataframe
print(df.count())
print(df.take(5))
df.head(5)

3
[Row(col='c1', value='v1'), Row(col='c2', value='v2'), Row(col='c3', value='v3')]


[Row(col='c1', value='v1'),
 Row(col='c2', value='v2'),
 Row(col='c3', value='v3')]

In [17]:
# check explain plan
df.explain("cost")

== Optimized Logical Plan ==
LogicalRDD [col#112, value#113], false, Statistics(sizeInBytes=8.0 EiB)

== Physical Plan ==
*(1) Scan ExistingRDD[col#112,value#113]




In [18]:
df.toJSON().first()

'{"col":"c1","value":"v1"}'

In [20]:
df.toPandas()

,col,value
0,c1,v1
1,c2,v2
2,c3,v3


In [21]:
# createOrReplaceTempView
df.createOrReplaceTempView("my_table")
spark.sql("select * from my_table").show()

# createOrReplaceGlobalTempView. Este comando permite partilhar o mesmo DataFrame em várias sessões.
# O anterior Local não permite isso.

df.createOrReplaceGlobalTempView("my_table2")
spark.sql("select * from globaL_temp.my_table2").show()


+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [23]:
spark.sql("select * from globaL_temp.my_table2 union all select * from globaL_temp.my_table2").show()
df.unionAll(df).show()

# os dois comandos fazem o mesmo. trabalhar com códigos orientados a objetos acaba por ser melhor

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



## write operations

- df.write.format("parquet").save(path)
- df.write.format("json").save(path)
- df.write.format("csv").save(path)

### if using delta.io
- df.write.format("delta").saveAsTable(table_name)

